# Model: Combined - LSTM for (merged) text + Dense for others

Our NN models vary in the way we preprocess the text features (most important feature) towards feeding it into our NN.
This also effects the architechture of the NN.

In this notebook we:
1. Merge all text features (Title, Description, params) into one feature.
2. We learn the representation of the text feature by an LSTM (its output vector is the text representation).
3. We combine the LSTMed text with all other features.

In [5]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
%run modules.ipynb

In [7]:
DATA_PATH = './data'
HELPER_DATA_PATH = './helper_data'

## Feature enrichment

In [8]:
# print('loading data...')
# train, test = load_data(DATA_PATH)
# train, test = basic_enrichment(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = load_image_features(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = load_text_features(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = add_aggregated_features(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = numeric_features_cleaning(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = complete_image_top_1(train, test, helper_data_path=HELPER_DATA_PATH)
# train, test = complete_price(train, test, helper_data_path=HELPER_DATA_PATH)

loading data...
Adding basic features...
Done adding basic features.
Adding image features...


/home/ifallach/avito/stephan_feature_enrichment.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[df[col] >= df[col].quantile(0.99)][col] = df[col].quantile(0.99)
/home/ifallach/avito/stephan_feature_enrichment.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[df[col] >= df[col].quantile(0.99)][col] = df[col].quantile(0.99)


Done loading image features.
Loading text features...
Done loading text features.
Loading aggregated features...
Done loading aggregated features.
Loading aggregated features...
Done loading aggregated features.
Cleaning and completing numeric features...


/home/ifallach/avito/stephan_feature_enrichment.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[df[col] >= df[col].quantile(0.99)][col] = df[col].quantile(0.99)
/home/ifallach/avito/stephan_feature_enrichment.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[df[col] >= df[col].quantile(0.99)][col] = df[col].quantile(0.99)


Done cleaning numeric features.
Completing image_top_1 features...
Done loading image_top_1 completions.
Completing price...
Done loading log_price_regression.


In [9]:
list(train.columns)

['item_id',
 'user_id',
 'region',
 'city',
 'parent_category_name',
 'category_name',
 'param_1',
 'param_2',
 'param_3',
 'title',
 'description',
 'price',
 'item_seq_number',
 'activation_date',
 'user_type',
 'image',
 'image_top_1',
 'deal_probability',
 'has_description',
 'has_price',
 'has_params',
 'has_image',
 'month',
 'day',
 'weekday',
 'user_ads_count',
 'title_description_params',
 'img_size',
 'img_sharpness',
 'img_luminance',
 'img_colorfulness',
 'img_confidence',
 'img_keypoints',
 'log_img_sharpness',
 'log_img_keypoints',
 'title_word_count',
 'description_non_regular_chars_ratio',
 'description_word_count',
 'merged_params_word_count',
 'description_sentence_count',
 'description_words/sentence_ratio',
 'title_capital_letters_ratio',
 'description_capital_letters_ratio',
 'title_non_regular_chars_ratio',
 'title_num_of_newrow_char',
 'description_num_of_newrow_char',
 'title_num_adj',
 'title_num_nouns',
 'title_adj_to_len_ratio',
 'title_noun_to_len_ratio',
 '

## Vectorize features towards input to an NN

In [10]:
# text_features = ['title', 'description', 'param123']
text_feature = 'title_description_params'
cat_features = ['user_type', \
                'region', 'city', \
                'parent_category_name', 'category_name', 'param_1', 'param_2', 'param_3', \
                'image_top_1_class', 'image_top_1_rounded_regression', \
                'month', 'day', 'weekday', \
                'has_price', 'has_description', 'has_params', 'has_image']
cont_ord_features = ['image_top_1_regression', \
                     'log_price_regression', \
                     'avg_days_up_user', 'avg_times_up_user', 'n_user_items', 'user_ads_count', \
                     'log_item_seq_number', \
                     'img_size', 'img_luminance', 'img_colorfulness', 'img_confidence', 'log_img_sharpness', 'log_img_keypoints', \
                     'title_word_count', 'description_word_count', 'merged_params_word_count', \
                     'description_non_regular_chars_ratio', 'title_capital_letters_ratio','description_capital_letters_ratio', \
                     'title_non_regular_chars_ratio', 'title_adj_to_len_ratio', 'title_noun_to_len_ratio',\
                     'title_sentiment']

train_y_prob = train['deal_probability']

## Preprocess text towards input to an LSTM

Import stopwords.

In [11]:
import codecs

stopwords = set()
with codecs.open(os.path.join(HELPER_DATA_PATH, 'stopwords_ru.txt'), encoding='cp1251') as ins:
    for w in ins:
        word = w.strip("\r\n")
        word = word.strip("\n")
        stopwords.add(word.lower())

Tokenize and Vectorize (keras encoded one-hot representation (each onehot vec represented as an int number)) text feature.

See: https://keras.io/preprocessing/text/#one_hot

In [12]:
from keras.preprocessing.text import one_hot, Tokenizer, text_to_word_sequence

# Those consts are important for the NN itself
max_words_in_desc_title_param = 150 # See text analysis notebook. 95% are shorter than a 110.
word_embed_dim = 300

train_x_text = train[[text_feature]]
test_x_text = test[[text_feature]]
train_x_text[text_feature] = train_x_text[text_feature].str.lower()
test_x_text[text_feature] = test_x_text[text_feature].str.lower()
tokenizer = Tokenizer(num_words = max_words_in_desc_title_param)
all_text = np.hstack([train_x_text[text_feature], test_x_text[text_feature]])

print('Tokenizing text...')
tokenizer.fit_on_texts(all_text)
print('Done tokenizing.')

del all_text
gc.collect()

/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Tokenizing text...
Done tokenizing.


0

In [13]:
# By default text_to_word_sequence automatically does 4 things:
#   Splits words by space (split=” “), 
#   Filters out punctuation (filters=’!”#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n’).
#   Converts text to lowercase (lower=True).
# We add stopwords filtering to the process.

def my_text_to_word_sequence(text):
    result = []
    for word in text_to_word_sequence(text):
        if word not in stopwords:
            result.append(word)
    return result

In [14]:
print('Applying tokenizer on text...')

train_x_text[text_feature] = train_x_text.apply(lambda r: [tokenizer.word_index[word] for word in my_text_to_word_sequence(r[text_feature])], axis=1)
test_x_text[text_feature] = test_x_text.apply(lambda r: [tokenizer.word_index[word] for word in my_text_to_word_sequence(r[text_feature])], axis=1)
    
train_x_text = pad_sequences(train_x_text[text_feature], maxlen=max_words_in_desc_title_param)
test_x_text = pad_sequences(test_x_text[text_feature], maxlen=max_words_in_desc_title_param)

print('Done.')

Applying tokenizer on text...


/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Done.


Import and load FastText (Facebook's) Russian wikipedia word embeddings.

In [15]:
def get_fasttext_embedding_matrix(data_path, tokenizer, embedding_dim):
    print('loading embeddings...')
    
    EMBEDDING_FILE_PATH = os.path.join(data_path, 'cc.ru.300.vec')
    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE_PATH))

    vocab_size = len(tokenizer.word_index) + 2
    embedding_matrix = np.random.rand(vocab_size, embedding_dim)
    
    print('creating embedding matrix...')
    embedding_exists = 0
    no_embeddings = 0
    for word, i in tokenizer.word_index.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
            embedding_exists += 1
        else:
            no_embeddings += 1
    
    print ("There are total of {} words in our corpus.".format(embedding_exists+no_embeddings))
    print ("There are {} embeddings in FastText.".format(len(embeddings_index)))
    print ("We have embeddings for {} words ({}% existing embeddings).".format(embedding_exists, \
                                                                               (100*embedding_exists/(embedding_exists+no_embeddings))))
    print ("Embedding is missing for {} words.".format(no_embeddings))

    del embeddings_index
    gc.collect()
    
    print('done loading embeddings...')
    return embedding_matrix, vocab_size

In [16]:
embedding_matrix, vocab_size = get_fasttext_embedding_matrix(data_path=DATA_PATH, \
                                                             tokenizer=tokenizer, embedding_dim = word_embed_dim)

loading embeddings...
creating embedding matrix...
There are total of 899487 words in our corpus.
There are 1888424 embeddings in FastText.
We have embeddings for 307819 words (34.221617433047946% existing embeddings).
Embedding is missing for 591668 words.
done loading embeddings...


#### Categorical features.

Vectorize all loaded categorical features.


See: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [17]:
train_x_cat = train[cat_features]
test_x_cat = test[cat_features]
for col in cat_features:
    train_x_cat[col] = train_x_cat[col].astype('category')
    test_x_cat[col] = test_x_cat[col].astype('category')

# Encode to integers.
# For vectorization (encoding) we concat both train and test into one
all_cat = pd.concat([train_x_cat, test_x_cat], axis = 0)
for col in cat_features:
    enc = preprocessing.LabelEncoder().fit(all_cat[col])
    train_x_cat[col] = enc.transform(train_x_cat[col])
    test_x_cat[col] = enc.transform(test_x_cat[col])

# One-hot encode:
enc = OneHotEncoder()
enc.fit(pd.concat([train_x_cat, test_x_cat], axis = 0))
train_x_cat = enc.transform(train_x_cat)
test_x_cat = enc.transform(test_x_cat)

del all_cat
gc.collect()

/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

63

#### Numerical(Continious/Ordinal) features

Normalize all loaded numeric features.

In [18]:
train_x_numerical = train[cont_ord_features]
test_x_numerical = test[cont_ord_features]
train_x_numerical.fillna(0, inplace = True)
test_x_numerical.fillna(0, inplace = True)
for col in cont_ord_features:
    train_x_numerical[col] = train_x_numerical[col].astype('float64')
    test_x_numerical[col] = test_x_numerical[col].astype('float64')

# Normalize features:
train_x_numerical = normalize(train_x_numerical, axis=0)
test_x_numerical = normalize(test_x_numerical, axis=0)

/home/ifallach/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ifallach/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

# Learning - Neural Net

Here we should have a pad sequence out of the text features
All categorials should be onehot encoded
All continious/ordinal should be vectorized

### Define network

TODO(): Try hidden_dim1 = 512, hidden_dim2 = 128 after loading FastText

In [19]:
lstm_out_dim = 256
cat_hidden_dim = 128
merged_hidden_dim = 256
out_dim = 1

Function that gets a tokenizer, and returns fasttext_embedding_matrix that will be loaded into the embedding layer of our NN

In [32]:
# LSTM
l_text_input = Input(shape=(max_words_in_desc_title_param,), name="text_lstm_input")
# Load FastText's weights:
l_text_embedding = Embedding(input_dim=vocab_size, output_dim=word_embed_dim, weights = [embedding_matrix], \
                             input_length=max_words_in_desc_title_param, name='text_embeddings')(l_text_input)
lstm_out = LSTM(lstm_out_dim, name='LSTM', dropout=0.2, recurrent_dropout=0.2)(l_text_embedding)

# Categoricals
l_cat_input = Input(shape=(train_x_cat.shape[1],), sparse=True, name="cat_input")
l_hidden_cat = Dense(cat_hidden_dim, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-6), name='l_hidden_cat')(l_cat_input)

# Numerical
l_numerical_input = Input(shape=(train_x_numerical.shape[1],), name="numerical_input")

# Aggregate all inputs into one hidden layer.
l_aggregative = concatenate([lstm_out, l_hidden_cat, l_numerical_input])

l_merged_hidden = Dense(merged_hidden_dim, activation='relu',
                            kernel_regularizer=regularizers.l2(1e-6), name='l_merged_hidden')(l_aggregative)

output = Dense(out_dim, activation='sigmoid', name='output')(l_merged_hidden) # This is the logistic regression output

rmsprop_opt = RMSprop(lr=0.0001) # Best for training RNNs.
def rmse_err(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

model = Model(inputs=[l_text_input, l_cat_input, l_numerical_input], outputs=output)
model.compile(optimizer=rmsprop_opt ,loss=[rmse_err]) # Higher weight for main output.
print(model.summary())

load = False
fname = 'NN-LSTM-COMBINED-MERGED-model-weights.h5'
if load:
    model.load_weights(fname)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_lstm_input (InputLayer)    (None, 150)          0                                            
__________________________________________________________________________________________________
text_embeddings (Embedding)     (None, 150, 300)     269846700   text_lstm_input[0][0]            
__________________________________________________________________________________________________
cat_input (InputLayer)          (None, 9950)         0                                            
__________________________________________________________________________________________________
LSTM (LSTM)                     (None, 256)          570368      text_embeddings[0][0]            
__________________________________________________________________________________________________
l_hidden_c

In [33]:
reduce_lr_cd = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
save_weights_cd = ModelCheckpoint(fname, save_weights_only=True, mode='auto', period=1)

nbatch_size = 512

model.fit({'text_lstm_input': train_x_text, 'cat_input': train_x_cat, 'numerical_input': train_x_numerical}, train_y_prob, 
          validation_split = 0.1, 
          epochs=4, 
          batch_size=nbatch_size,
          callbacks=[reduce_lr_cd, save_weights_cd])

# Train on all data
model.fit({'text_lstm_input': train_x_text, 'cat_input': train_x_cat, 'numerical_input': train_x_numerical}, train_y_prob, 
          epochs=2, 
          batch_size=nbatch_size,
          callbacks=[save_weights_cd])

/home/ifallach/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 269846700 elements. This may consume a large amount of memory.
  num_elements)


Train on 1353081 samples, validate on 150343 samples
Epoch 1/4
1353081/1353081 [==============================] - 14129s 10ms/step - loss: 0.2335 - val_loss: 0.2300
Epoch 2/4
 972800/1353081 [====================>.........] - ETA: 1:12:32 - loss: 0.2295

KeyboardInterrupt: 

### Save predictions to futher ensambels

In [ ]:
y_pred_train = model.predict({'text_lstm_input': train_x_text, 'cat_input': train_x_cat, 'numerical_input': train_x_numerical})
y_pred_test = model.predict({'text_lstm_input': test_x_text, 'cat_input': test_x_cat, 'numerical_input': test_x_numerical})

In [ ]:
res_train_df = pd.DataFrame(train['item_id'])
res_train_df['deal_probability'] = y_pred_train

In [ ]:
res_test_df = pd.DataFrame(test['item_id'])
res_test_df['deal_probability'] = y_pred_test

In [ ]:
res_df = pd.concat([res_train_df, res_test_df], axis = 0)

In [ ]:
res_df.to_csv('./predictions/NN-LSTM-COMBINED-MERGED.csv.gz', index=False, compression='gzip')